In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel
from helical.models.genept.genept_config import GenePTConfig
import torch
import json
from tqdm import tqdm
from datasets import Dataset
import torch
import numpy as np


/home/maxime/miniconda3/envs/helical_clean/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.5.1 available.
INFO:datasets:TensorFlow version 2.17.0 available.
2024-12-19 19:34:31.888489: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 19:34:31.897573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 19:34:31.907418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN fact

In [2]:
default_configurer = GenePTConfig(device="cuda")
config  = default_configurer.config
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
model = AutoModel.from_pretrained(config["model_name"],torch_dtype=torch.bfloat16,attn_implementation="sdpa").to(config["device"])

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.91it/s]


In [3]:
all_texts = json.load(open("./NCBI_cleaned_summary_of_genes.json"))

In [4]:
df = pd.DataFrame([all_texts]).T
df.columns = ["text"]

In [5]:
df['embeddings'] = 0

In [9]:
batch_size = 2
embeddings = []
for batch in tqdm(range(0,len(df),batch_size)):
    input_ids = tokenizer(df['text'][batch:batch+batch_size].values.tolist(),return_tensors="pt",max_length=1000,padding=True,truncation=True).to(config["device"])
    res = model(input_ids['input_ids'],attention_mask=input_ids['attention_mask'],output_hidden_states=True)
    # print(res.last_hidden_state.shape)
    # print(res.last_hidden_state[:,-1].float().cpu().detach().numpy().shape)
    embeddings.append(res.last_hidden_state[:,-1].float().cpu().detach().numpy())
    del input_ids
    del res

100%|██████████| 16852/16852 [27:36<00:00, 10.17it/s]


In [14]:
all_embs = np.vstack(embeddings)
np.save("genept_embeddings.npy",all_embs)
df['embeddings'] = all_embs.tolist()

In [37]:
with open("genept_embeddings.json","w") as f:
    json.dump(df.to_dict(orient="index"),f)

In [1]:
from helical.models.genept.model import GenePT, GenePTConfig
from datasets import load_dataset
from helical.utils import get_anndata_from_hf_dataset

genept_config = GenePTConfig(device="cuda")
genept = GenePT(configurer = genept_config)

# either load via huggingface
hf_dataset = load_dataset("helical-ai/yolksac_human",split="train[:5%]", trust_remote_code=True, download_mode="reuse_cache_if_exists")
ann_data = get_anndata_from_hf_dataset(hf_dataset)

INFO:datasets:PyTorch version 2.5.1 available.
INFO:datasets:TensorFlow version 2.17.0 available.
2024-12-19 19:48:07.714274: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 19:48:07.723481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 19:48:07.733196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 19:48:07.736128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one h

In [2]:
data = genept.process_data(ann_data)

In [ ]:
embs = genept.get_embeddings(data)

In [5]:
embs.shape

torch.Size([1267, 4096])